## Notes

- Camera CS: Z - forward, X - right, Y - down

## More datasets

- http://www.cvlibs.net/datasets/karlsruhe_sequences/

## Poses transormation

P2 = T * P1
T = P2 * P1^-1


In [ ]:
# Old transformation
def get_transofrmation(pose1, pose2):
    rot_pose1 = pose1[:3,:3]
    rot_pose2 = pose2[:3,:3]
    inv_rot_pose1 = np.linalg.inv(rot_pose1)
    inv_rot_pose2 = np.linalg.inv(rot_pose2)
    trans_pose1 = pose1[:,3]
    trans_pose2 = pose2[:,3]
    
    r2_invr1 = rot_pose2.dot(inv_rot_pose1)
    trans = trans_pose2 - (r2_invr1.dot(trans_pose1))
    rot = r2_invr1
    
    trans = trans[..., np.newaxis]
    return np.hstack((rot, trans))

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os
import pprint 
pp = pprint.PrettyPrinter(indent=4)

from scipy.spatial.transform import Rotation as R

ROOT_DIR = '../'

In [ ]:
DATASET_DIR = os.path.join(ROOT_DIR, 'data/KITTY/dataset')
POSES_DIR = os.path.join(DATASET_DIR, 'poses')
SEQUENCE_IDX = '00'
SEQUENCE_DIR = os.path.join(DATASET_DIR, 'sequences', SEQUENCE_IDX)

In [ ]:
POSE_PATH = os.path.join(POSES_DIR, f'{SEQUENCE_IDX}.txt')
TIMES_PATH = os.path.join(SEQUENCE_DIR, 'times.txt')

poses_data = np.fromfile(POSE_PATH, sep=' ')
poses_data = poses_data.reshape((-1, 3, 4))

# Convert to 4x4 matrices
last_row = np.array([[[0,0,0,1]]])
last_rows = np.repeat(last_row, axis=0, repeats=poses_data.shape[0])
poses_data = np.hstack((poses_data, last_rows))

times_data = np.fromfile(TIMES_PATH, sep='\n')

In [ ]:
times_data.shape, poses_data.shape

In [ ]:
pose_0 = poses_data[0]
pose_0

In [ ]:
pose_1 = poses_data[1]
pose_1

In [ ]:
pose_10 = poses_data[10]
pose_10

In [ ]:
def get_transofrmation(pose1, pose2):
    # Poses are othogonal!
    return pose2 @ np.linalg.inv(pose1)

In [ ]:
transform = get_transofrmation(poses_data[0], poses_data[1])
transform

In [ ]:
pose_1, pose_10

In [ ]:
T1 = pose_10 @ np.linalg.inv(pose_1)
T2 = np.linalg.inv(pose_1) @ pose_10

T1, T2, abs(T1 - T2) > 1e-5

In [ ]:
P10 = T1 @ pose_1
P10, abs(P10 - pose_10) > 1e-5

In [ ]:
P10 = pose_1 @ T2
P10, abs(P10 - pose_10) > 1e-5

In [ ]:
r = R.from_matrix(transform[:3,:3])
r.as_quat()

In [ ]:
r.as_euler('zyx', degrees=True)

In [ ]:
poses_data[90], poses_data[100]

In [ ]:
turn = get_transofrmation(poses_data[90], poses_data[100])
r = R.from_matrix(turn[:3,:3])
r.as_euler('zyx', degrees=True)

In [ ]:
turn[:3,3]

In [ ]:
turn @ poses_data[90] 

In [ ]:
points = []
for pose in poses_data[:100]:
    points.append(
        (pose[0,3], pose[1,3], pose[2,3])
    )
    
points = np.array(points) 
plt.figure(figsize=(9,9))
plt.plot(points[:,0], points[:,2])

# Test metrics

In [ ]:
import sys
sys.path.append('..')

from shared import metrics

In [ ]:
gt_poses = poses_data.copy()
pred_poses = poses_data.copy()

In [ ]:
diffs = np.diff(gt_poses[:,:3,3], axis=0)
deltas = np.linalg.norm(diffs, axis=1)
dists = np.cumsum([0, *deltas])

In [ ]:
ate = metrics.compute_ATE(gt_poses, pred_poses)
ate

In [ ]:
rpe_trns, rpe_rot = metrics.compute_RPE(gt_poses, pred_poses)
rpe_trns, rpe_rot

In [ ]:
def plot_trajectory(poses_gt, poses_result):
    plot_keys = ["Ground Truth", "Ours"]
    fontsize_ = 15

    poses_dict = {}
    poses_dict["Ground Truth"] = poses_gt
    poses_dict["Ours"] = poses_result

    fig = plt.figure()
    ax = plt.gca()
    ax.set_aspect('equal')

    for key in plot_keys:
        pos_xz = []
        frame_idx_list = range(len(poses_dict["Ours"]))
        for frame_idx in frame_idx_list:
            # pose = np.linalg.inv(poses_dict[key][frame_idx_list[0]]) @ poses_dict[key][frame_idx]
            pose = poses_dict[key][frame_idx]
            pos_xz.append([pose[0, 3],  pose[2, 3]])
        pos_xz = np.asarray(pos_xz)
        plt.plot(pos_xz[:, 0],  pos_xz[:, 1], label=key)

    plt.legend(loc="upper right", prop={'size': fontsize_})
    plt.xticks(fontsize=fontsize_)
    plt.yticks(fontsize=fontsize_)
    plt.xlabel('x (m)', fontsize=fontsize_)
    plt.ylabel('z (m)', fontsize=fontsize_)
    plt.grid()
    plt.show()

In [ ]:
plot_trajectory(gt_poses, pred_poses)

In [ ]:
seq_errs = metrics.calc_sequence_errors(gt_poses, pred_poses)

avg_segment_errs = metrics.compute_segment_error(seq_errs)
pp.pprint(avg_segment_errs)

ave_r_err, ave_t_err = metrics.compute_overall_err(seq_errs)
trns_err_perc = ave_t_err*100
rot_err_deg_100m = ave_r_err/np.pi*180*100
print("Translational error (%): ", trns_err_perc)
print("Rotational error (deg/100m): ", rot_err_deg_100m)